## Running, Visualizing, and Analyzing Molecular Dynamics (MD) Simulations ##
In this notebook, you will use Python tools to play around with Molecular Dynamics (MD) simulations. In this example, we will prepare a system of interacting hexamers that may spontaneously self-assemble into hexameric sheets. The hexamers look like the following:

<div>
<img src="toy_hex.png" width="250"/>
</div>
   
where the cyan spheres are the body of the molecule and the pink spheres are virtual interaction sites that will allow each hexamer to bind to other hexamers. This interaction is attractive and has the following Gaussian functional form:

$$ U(r_{ij}) = -A \exp{(-B r_{ij})} $$

where $U$ is the potential energy in kcal/mol, $r_{ij}$ is the pair distance between sites in Angstroms, and $A$/$B$ are model parameters. 

All of our simulations will be performed using LAMMPS, an open-source MD package developed at Sandia National Lab.

## What am I looking at right now?
This is a "Jupyter notebook", a tool for using Python interactively. For example, you can quickly work with data, run simulations, visualize data, etc. then decide you want to see how your results might change if you change a parameter earlier in your workflow. If you've used Mathematica before, the idea behind a Jupyter notebook is very similar. You hit `shift-enter` or `shift-return` to execute the code in a "cell". 

**Beware:**
- the good thing about notebooks is that they let you interact with your data in very flexible ways
- the bad thing is that you can execute cells out of order and overwrite variables in ways you forget or didn't expect

If you're getting weird results, it's best to either do `Cell->Run All` at the top to reset the entire notebook, or if really needed, `Kernel->Restart`

Let us run our first cell (right below) by hitting `shift-enter` inside the cell. This cell will import Python modules that we will need for the rest of the exercise. Note that the left of the cell will have a "star" marker (i.e., "In \[*\]") when it is running and an integer (i.e., "In \[1\]")when it is done.

In [1]:
# Time to setup the notebook! This might take a few minutes to finish.
# We are importing modules/libraries that other folks have created
# These libraries contain functions that will use (so that we don't reinvent the wheel)

import numpy as np
import mdtraj as md
import nglview as ngl
from lammps import IPyLammps

### Preparing the LAMMPS simulation ###
In your folder, you have the following files: 

(1) **supercell.pdb** - this is a file format known as the 'Protein Data Bank' format and contains the initial coordinates of our system as well as topology information (names and indices of atoms, residues, etc.)

(2) **system.data** - this is an input file used by LAMMPS and contains the coordinates and topology information for a single hexamer, which we will later tile to form the complete system

(3) **input.setup** - this is an input file used by LAMMPS that determines most of the simulation parameters (force field definition, integration method, timestep, etc.)

All of these files are text files so you should feel free to view their contents using your favorite text editor.

In the next cell, we will prepare the LAMMPS simulation using our input files. We can further modify the simulation using the `lammps` module, as you will see.

In [2]:
L = IPyLammps() # this prepares the lammps Python object
L.variable("GAUSSA equal 2.65") # this sets the "A" parameter in the Gaussian to 2.65 kcal/mol
L.file("input.setup") # this loads in the other simulation settings


LAMMPS output is captured by PyLammps wrapper
LAMMPS (23 Jun 2022 - Update 1)
OMP_NUM_THREADS environment is not set. Defaulting to 1 thread. (src/comm.cpp:98)
  using 1 OpenMP thread(s) per MPI task


### Running the simulation ###

Now that the simulation is prepared, we can run the simulation. Let us run a short simulation of 25000 steps to see what happens. You'll see that a file called `md.dcd` will be created.

NOTE: This next cell make take a few minutes to run. You will know it is still busy if you see the star in "In \[*\]"

In [3]:
L.run(25000) #run for 25000 steps, should take a few seconds to run

Reading data file ...
  orthogonal box = (-10 -10 -10) to (10 10 10)
  1 by 1 by 1 MPI processor grid
  reading atoms ...
  27 atoms
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:    0        0        0       
  special bond factors coul:  0        0        0       
     0 = max # of 1-2 neighbors
     0 = max # of 1-3 neighbors
     0 = max # of 1-4 neighbors
     1 = max # of special neighbors
  special bonds CPU = 0.000 seconds
  read_data CPU = 0.018 seconds
Replicating atoms ...
  orthogonal box = (-10 -10 -10) to (50 50 50)
  1 by 1 by 1 MPI processor grid
  729 atoms
Finding 1-2 1-3 1-4 neighbors ...
  special bond factors lj:    0        0        0       
  special bond factors coul:  0        0        0       
     0 = max # of 1-2 neighbors
     0 = max # of 1-3 neighbors
     0 = max # of 1-4 neighbors
     1 = max # of special neighbors
  special bonds CPU = 0.000 seconds
  replicate CPU = 0.001 seconds
  create bodies CPU = 0.001 seconds
  27 rigid bodies wit

['Generated 0 of 3 mixed pair_coeff terms from geometric mixing rule',
 'Neighbor list info ...',
 '  update every 1 steps, delay 1 steps, check yes',
 '  max neighbors/atom: 2000, page size: 100000',
 '  master list distance cutoff = 10',
 '  ghost atom cutoff = 10',
 '  binsize = 5, bins = 12 12 12',
 '  3 neighbor lists, perpetual/occasional/extra = 3 0 0',
 '  (1) pair soft, perpetual, skip from (3)',
 '      attributes: half, newton on',
 '      pair build: skip',
 '      stencil: none',
 '      bin: none',
 '  (2) pair gauss, perpetual, skip from (3)',
 '      attributes: half, newton on',
 '      pair build: skip',
 '      stencil: none',
 '      bin: none',
 '  (3) neighbor class addition, perpetual',
 '      attributes: half, newton on',
 '      pair build: half/bin/newton',
 '      stencil: half/bin/3d',
 '      bin: standard',
 'Setting up Verlet run ...',
 '  Unit style    : real',
 '  Current step  : 0',
 '  Time step     : 10',
 'Per MPI rank memory allocation (min/avg/ma

### Visualizing the trajectory ###

You'll see in the output of the previous block that a bunch of text was printed out. This is the "log" of the simulation and contains useful information. For our purposes, we can skip this and instead visualize the actual trajectory.

We will use `nglview` to display the trajectory. Once it loads, you can click play to see the atoms in motion!

In [5]:
traj = md.load("md.dcd", top="supercell.pdb")
view = ngl.show_mdtraj(traj)
view.clear_representations()

# show all sites as either cyan or pink
view.add_spacefill(selection=".1 or .2", color="cyan", radius=3.0)
view.add_spacefill(selection=".3", color="pink", radius=3.0)

# add to output
view

NGLWidget(max_frame=25)

### Extending the simulation ###

Let us extend the simulations now. Using the current parameters, we should expect to see self-assembly after about 1,000,000 (1M or 1e6) steps. 

NOTE: This next cell make take a few minutes to run. You will know it is still busy if you see the star in "In \[*\]"

In [6]:
L.run(1000000) #run for 1M steps, should take a few minutes to run (80 sec on my machine)

['Generated 0 of 3 mixed pair_coeff terms from geometric mixing rule',
 'Setting up Verlet run ...',
 '  Unit style    : real',
 '  Current step  : 25000',
 '  Time step     : 10',
 'Per MPI rank memory allocation (min/avg/max) = 12.82 | 12.82 | 12.82 Mbytes',
 '   Step         S/CPU           Temp           f_1           KinEng         PotEng         TotEng         E_pair           Lx             Ly             Lz            Pxx            Pyy            Pzz      ',
 '     25000   0              276.43042      271.31133      43.671288     -25.356243      18.315045     -25.356243      60             60             60             9.0449327      9.2706221      9.4062472    ',
 '     50000   18455.977      279.54641      274.36962      44.163561     -32.762679      11.400882     -32.762679      60             60             60            -372.61076     -22.837494      349.64033    ',
 '     75000   20103.286      289.56266      284.20039      45.745958     -8.98332        36.762638     -8

In [7]:
traj = md.load("md.dcd", top="supercell.pdb")
view = ngl.show_mdtraj(traj)
view.clear_representations()

# show all sites as either cyan or pink
view.add_spacefill(selection=".1 or .2", color="cyan", radius=3.0)
view.add_spacefill(selection=".3", color="pink", radius=3.0)

# add to output
view

NGLWidget(max_frame=1025)

### Observation debrief ###

What did you observe when you played those movies? Did you see aggregation of the hexamers into a cluster? What shape is the cluster?

Keep this behavior in mind because now we are going to adjust the model parameters to see what happens. You should change the $A$ parameter to be lower (e.g., 2.0 kcal/mol) or higher (e.g., 4.0 kcal/mol). To do so, go back to the top of the notebook and change the line that originally says:

`L.variable("GAUSSA equal 2.65")`

Note that the original simulation uses 2.65 kcal/mol. What do you think will happen in either scenario? Do you notice any changes in assembly behavior? 